In [1]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from src.dh import DHFrame
import src.tools as tools
import sympy as sp


In [8]:

r1, r2, r3, r4, a1, a2 = 0, 0.400, 0.455, 10**(-3)*np.sqrt(420**2+35**2), 0.025, 0.035
køddevinkel = np.arctan2(35, 420)

T_SB = np.array([[0,  1,   0, 0],
                 [0,  0,  -1, -(0.420+0.025+0.080)],
                 [-1, 0,   0, 0.89],
                 [0,  0,   0, 1]])

#print('forventer at P_W er: \n', tools.pointFromT(tools.makeT_SW(T_SB)))

sp.Matrix((np.around(tools.inverseKinematicsTheta123(T_SB),1)))

theta = np.pi/180*np.array([-45, -90, 90, 0, 0, 0])
 
kr6 = KR6(theta)

T = kr6.Tsb_IK

sp.Matrix(np.around(T,2))

# T = np.array([[0,  -0.5,   -0.5, 0.371231],
#               [0,   0.5,    0,   0.371231],
#               [1,    0,   -0.5, 890],
#               [0,     0,    0,   1]])

# print(tools.pointFromT(T))

# sp.Matrix(T)



print(sp.Matrix((np.around(tools.inverseKinematicsTheta123(T),1))))
tools.inverseKinematicsTheta456(T)


kr6=KR6(np.pi/180*np.array([-45, -90, 90, 0, 0, 0]))

S_1to3 = np.array([kr6._Slist[0], kr6._Slist[1], kr6._Slist[2]])

print(S_1to3)

sp.Matrix(tools.inverseKinematicsTheta456(T, S_1to3, ))

Matrix([[-45.0000000000000, -8.80000000000000, -80.5000000000000], [-45.0000000000000, -90.0000000000000, 90.0000000000000], [135.000000000000, -171.300000000000, 83.3000000000000], [135.000000000000, -96.3000000000000, -73.8000000000000]])
90.0
[[ 0.  0.  0. -1.  0. -1.]
 [ 0.  1.  1.  0.  1.  0.]
 [-1.  0.  0.  0.  0.  0.]]


In [3]:
config_fasit = np.array([0,-np.pi/4, np.pi/6, 0,0,0])
config2 = np.array([0, -np.pi/2, np.pi/2, 0, 0, 0])
config3 = np.array([0, np.pi/2, 0, 0, 0, 0])




def test_a_IK(config_fasit):

    kr6_fasit = KR6(config_fasit)

    P_SW_fasit = tools.pointFromT(kr6_fasit.Tsw) # define the point of the wrist

    thetalist = kr6_fasit.aIK_thetaLists # Fetching the analytical solution(s)

    for el in thetalist:
        config_test = np.array([el[0], el[1], el[2], 0,0,0])

        kr6_test = KR6(config_test)

        P_SW_test = tools.pointFromT(kr6_test.Tsw)

        print(kr6_fasit.Tsb_poe)
        # print(kr6_fasit.Tsw)
        print(P_SW_test)
        print(P_SW_fasit)

        print(np.allclose(P_SW_fasit, P_SW_test, 0.01, 0.01))

print(test_a_IK(np.array([0, 0, 0, 0, 0, 0])))

# kr6.add_FK_to_viewer(viewer)

# print(f"Number of iteration: {kr6.IK[3]}")

# kr6.add_nIK_to_viewer(viewer)
viewer = Viewer(background="white")
viewer.add(Grid())
KR6(config_fasit).add_FK_to_viewer(viewer)


c:\Users\test\git\TPK4170_project-1\src\tools.py:160: RuntimeWarning: invalid value encountered in sqrt
  s3_pos = np.sqrt(1-c3**2)
c:\Users\test\git\TPK4170_project-1\src\tools.py:161: RuntimeWarning: invalid value encountered in sqrt
  s3_neg = -np.sqrt(1-c3**2)


[[ 0.     0.     1.     0.98 ]
 [ 0.    -1.     0.     0.   ]
 [ 1.     0.     0.     0.435]
 [ 0.     0.     0.     1.   ]]
[0.9   0.    0.435]
[0.9   0.    0.435]
True
[[ 0.     0.     1.     0.98 ]
 [ 0.    -1.     0.     0.   ]
 [ 1.     0.     0.     0.435]
 [ 0.     0.     0.     1.   ]]
[0.02869285 0.         0.36559827]
[0.9   0.    0.435]
False
[[ 0.     0.     1.     0.98 ]
 [ 0.    -1.     0.     0.   ]
 [ 1.     0.     0.     0.435]
 [ 0.     0.     0.     1.   ]]
[nan nan nan]
[0.9   0.    0.435]
False
[[ 0.     0.     1.     0.98 ]
 [ 0.    -1.     0.     0.   ]
 [ 1.     0.     0.     0.435]
 [ 0.     0.     0.     1.   ]]
[nan nan nan]
[0.9   0.    0.435]
False
None


C:\Users\test\anaconda3\envs\tpk4170_venv\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…